In [ ]:
# %pip install pandas
# %pip install numpy
# %pip install networkx
# %pip install scikit-learn

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
from coordinatedActivity.coRetweet import *

In [10]:
root_dir = "/project/ll_774_951/InfoOpsNationwiseDriverControl"
countries_dir = os.listdir("/project/ll_774_951/InfoOpsNationwiseDriverControl")

In [5]:
print(countries_dir)

['qatar', 'ghana', 'tanzania', 'mexico', 'venezuela_set2', 'spain', 'ecuador', 'catalonia', 'egypt', 'armenia', 'china', 'venezuela', 'cuba', 'uganda', 'turkey', 'uae', 'russia', 'bangladesh', 'thailand', 'iran']


In [6]:
if(os.path.isdir("cache") == False):
    os.mkdir("./cache")

In [7]:
import gzip

def open_gz_csv(file_dir):
    with gzip.open(file_dir) as f:
        df = pd.read_csv(f,low_memory=False)
    return df

def open_gz_jsonl(file_dir):
    with gzip.open(file_dir,"rb") as f:
        d = f.read()
    
        data = [json.loads(line.decode().strip()) for line in d]
    df = pd.DataFrame(data)
    print(df.head(3))
    return df

In [11]:
import json

# faulty_countries = ["venezuela_set2","egypt"]
# countries_dir = list(set(countries_dir).difference(set(faulty_countries)))
for country in countries_dir:
    print(country)
    files_dir = os.listdir(os.path.join(root_dir,country))

    ## Country File Names Check
    control_check = list(filter(lambda x:"control_driver_tweets" in x,files_dir))
    treated_check = list(filter(lambda x:"tweets_csv_unhashed" in x,files_dir))
    
    if(len(control_check) >= 1 and len(treated_check) >= 1):
        dataset_dirs.append(os.path.join(root_dir,country,treated_check[0]))
        dataset_dirs.append(os.path.join(root_dir,country,control_check[0]))

qatar
ghana
tanzania
mexico
venezuela_set2
spain
ecuador
catalonia
egypt
armenia
china
venezuela
cuba
uganda
turkey
uae
russia
bangladesh
thailand
iran


In [12]:
print(dataset_dirs)

['/project/ll_774_951/InfoOpsNationwiseDriverControl/ecuador/ecuador_082019_tweets_csv_unhashed.csv.gz', '/project/ll_774_951/InfoOpsNationwiseDriverControl/ecuador/control_driver_tweets_ecuador_082019.jsonl.gz', '/project/ll_774_951/InfoOpsNationwiseDriverControl/ecuador/ecuador_082019_tweets_csv_unhashed.csv.gz', '/project/ll_774_951/InfoOpsNationwiseDriverControl/ecuador/control_driver_tweets_ecuador_082019.jsonl.gz', '/project/ll_774_951/InfoOpsNationwiseDriverControl/qatar/qatar_082020_tweets_csv_unhashed.csv', '/project/ll_774_951/InfoOpsNationwiseDriverControl/qatar/control_driver_tweets_qatar_082020.jsonl.gz', '/project/ll_774_951/InfoOpsNationwiseDriverControl/ghana/unhashed_2020_03_ghana_nigeria_032020_tweets_csv_unhashed.csv.gz', '/project/ll_774_951/InfoOpsNationwiseDriverControl/ghana/control_driver_tweets_ghana_nigeria_032020.jsonl.gz', '/project/ll_774_951/InfoOpsNationwiseDriverControl/spain/spain_082019_tweets_csv_unhashed.csv.gz', '/project/ll_774_951/InfoOpsNationwis

In [13]:
def GenerateDatasets(datasetsPaths, outputDir):
    for file in datasetsPaths:
        if 'control' in file:
            if file[-2:] == 'gz':
                try:
                    with gzip.open(file) as f:
                        control = pd.concat([control, get_negative_data(pd.read_json(f, lines =True)[['id', 'full_text', 'lang', 'user', 'created_at']])])
                except:
                    with gzip.open(file) as f:
                        control = get_negative_data(pd.read_json(f, lines =True)[['id', 'full_text', 'lang', 'user', 'created_at']])
            else:
                try:
                    with open(file) as f:
                        control = pd.concat([control, get_negative_data(pd.read_json(f, lines =True)[['id', 'full_text', 'lang', 'user', 'created_at']])])
                except:
                    with open(file) as f:
                        control = get_negative_data(pd.read_json(f, lines =True)[['id', 'full_text', 'lang', 'user', 'created_at']])
        else:
            if file[-2:] == 'gz':
                try:
                    with gzip.open(file) as f:
                        treated = pd.concat([treated, get_positive_data(pd.read_csv(f))])
                except:
                    with gzip.open(file) as f:
                        treated = get_positive_data(pd.read_csv(f))
            else:
                try:
                    with open(file) as f:
                        treated = pd.concat([treated, get_positive_data(pd.read_csv(f))])
                except:
                    with open(file) as f:
                        treated = get_positive_data(pd.read_csv(f))
    
            
    pos_en_df_all = preprocess_text(treated)
    del treated
    neg_en_df_all = preprocess_text(control)
    del control
    
    pos_en_df_all['tweet_text']  = pos_en_df_all['tweet_text'].replace(',', '')
    neg_en_df_all['tweet_text']  = neg_en_df_all['tweet_text'].replace(',', '')
    
    pos_en_df_all['clean_tweet'] = pos_en_df_all['tweet_text'].astype(str).apply(lambda x: msg_clean(x))
    neg_en_df_all['clean_tweet'] = neg_en_df_all['tweet_text'].astype(str).apply(lambda x: msg_clean(x))
    
    pos_en_df_all = pos_en_df_all[pos_en_df_all['clean_tweet'].apply(lambda x: len(x.split(' ')) > 4)]
    neg_en_df_all = neg_en_df_all[neg_en_df_all['clean_tweet'].apply(lambda x: len(x.split(' ')) > 4)]

    pos_en_df_all['tweet_time'] = pos_en_df_all['tweetid'].apply(lambda x: get_tweet_timestamp(x))
    neg_en_df_all['tweet_time'] = neg_en_df_all['tweetid'].apply(lambda x: get_tweet_timestamp(x))

    pos_en_df_all.to_csv("/scratch1/ashwinba/consolidated/treated_consolidated.csv")
    neg_en_df_all.to_csv("/scratch1/ashwinba/consolidated/control_consolidated.csv")